# New AIDA-X Model Trainer for Colab
This notebook will guide you through training a new model using the `new_model_trainer`.

**Instructions:**
1.  Make a copy of this notebook to your Google Drive (`File > Save a copy in Drive`).
2.  Follow the steps in each section, executing the code cells by pressing Shift+Enter or clicking the play button.
3.  Ensure your audio data and configuration files are stored in Google Drive for easy access.
4.  Adjust parameters using the form fields provided in the cells.

## 1. Setup Environment
This section clones the project repository (if you're running this notebook directly from GitHub or a fresh Colab instance), installs necessary dependencies from `requirements.txt`, checks for GPU availability, and mounts your Google Drive for data access.

In [ ]:
#@markdown ### (RUN CELL) 1.1: Initial Setup, Dependencies, and Google Drive
#@markdown This cell will:
#@markdown 1. Clone the repository (if not already in the correct environment).
#@markdown 2. Change directory to `new_model_trainer`.
#@markdown 3. Install required Python packages from `requirements.txt`.
#@markdown 4. Check for GPU and mount Google Drive.

import os
import subprocess
import sys

# --- Repository Setup ---
REPO_NAME = "Automated-GuitarAmpModelling" # The name of the repository
COLAB_CONTENT_PATH = "/content"
REPO_PATH = os.path.join(COLAB_CONTENT_PATH, REPO_NAME)
NEW_TRAINER_DIR_NAME = "new_model_trainer" # The specific subdirectory we need to be in
TARGET_DIR = os.path.join(REPO_PATH, NEW_TRAINER_DIR_NAME)

print(f"Expected final directory: {TARGET_DIR}")

if os.getcwd() == TARGET_DIR:
    print(f"Already in the correct directory: {os.getcwd()}")
    print("Pulling latest changes from repository...")
    result = subprocess.run(["git", "pull"], capture_output=True, text=True)
    print(result.stdout)
    if result.stderr and result.returncode != 0 : print(f"Git pull stderr: {result.stderr}")
elif os.path.exists(TARGET_DIR):
    print(f"Repository '{REPO_NAME}' and subdirectory '{NEW_TRAINER_DIR_NAME}' found.")
    os.chdir(TARGET_DIR)
    print(f"Changed directory to: {os.getcwd()}")
    print("Pulling latest changes from repository...")
    result = subprocess.run(["git", "pull"], capture_output=True, text=True)
    print(result.stdout)
    if result.stderr and result.returncode != 0 : print(f"Git pull stderr: {result.stderr}")
else:
    print(f"Cloning repository '{REPO_NAME}' into {COLAB_CONTENT_PATH}...")
    GIT_CLONE_URL = "https://github.com/AidaDSP/Automated-GuitarAmpModelling.git"
    
    # Clone outside the target dir first, then cd
    if os.path.exists(REPO_PATH):
        print(f"Repository path {REPO_PATH} already exists but not in TARGET_DIR. Removing and re-cloning for clean state.")
        subprocess.run(["rm", "-rf", REPO_PATH], check=True)
        
    result = subprocess.run(["git", "clone", GIT_CLONE_URL, REPO_PATH], capture_output=True, text=True)
    if result.returncode == 0:
        print("Repository cloned successfully.")
        os.chdir(TARGET_DIR)
        print(f"Changed directory to: {os.getcwd()}")
    else:
        print(f"ERROR: Cloning repository failed!\nStdout:\n{result.stdout}\nStderr:\n{result.stderr}")
        sys.exit("Failed to clone repository.")

# --- Install Dependencies ---
print("\nInstalling dependencies from requirements.txt...")
requirements_path = "requirements.txt"
if os.path.exists(requirements_path):
    try:
        # Ensure pip is up-to-date
        subprocess.run([sys.executable, "-m", "pip", "install", "--upgrade", "pip"], check=True, capture_output=True, text=True)
        # Install requirements
        result = subprocess.run([sys.executable, "-m", "pip", "install", "-r", requirements_path], check=True, capture_output=True, text=True)
        print("Dependencies installed successfully.")
        # print(result.stdout) # Optionally print stdout for installed packages
    except subprocess.CalledProcessError as e:
        print(f"ERROR: Installing dependencies failed!\nExit Code: {e.returncode}\nStdout:\n{e.stdout}\nStderr:\n{e.stderr}")
        sys.exit("Failed to install dependencies.")
else:
    print(f"ERROR: {requirements_path} not found in {os.getcwd()}. Cannot install dependencies.")
    sys.exit("requirements.txt not found.")

# --- PyTorch/GPU Check ---
print("\nChecking PyTorch and GPU availability...")
try:
    import torch
    print(f"PyTorch version: {torch.__version__}")
    if torch.cuda.is_available():
        print(f"CUDA is available. GPU: {torch.cuda.get_device_name(0)}")
    elif torch.backends.mps.is_available():
        print("MPS is available for PyTorch (Apple Silicon GPU).")
    else:
        print("No GPU (CUDA or MPS) found by PyTorch. Training will use CPU.")
except ImportError:
    print("ERROR: PyTorch is not installed. Please ensure dependencies were installed correctly.")
    sys.exit("PyTorch not found after installation attempt.")

# --- Mount Google Drive ---
print("\nMounting Google Drive at /content/drive...")
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    print("Google Drive mounted successfully.")
except ImportError:
    print("Could not import Google Drive module. This notebook is intended to be run in Google Colab.")
except Exception as e:
    print(f"An error occurred while mounting Google Drive: {e}")

print("\n--- Setup cell complete. --- ")

## 2. Data Preparation

This cell handles the setup of your training data. It will:
1. Ask for the path to your main data directory in Google Drive.
2. Ask for the name of your JSON training configuration file (which should be inside the directory specified above).
3. Copy your JSON configuration to Colab's local storage.
4. Parse this JSON config to find `dataset_items`.
5. For each item, copy the specified `input_path` and `target_path` audio files from Google Drive to Colab's local storage.
6. Rewrite the paths in `dataset_items` (and `output_dir`) in the configuration to point to these local Colab paths.
7. Save this modified configuration as `effective_training_config.json` in Colab, which will be used by the training script.

**Important:**
- Your main JSON configuration file (e.g., `my_run_config.json`) must be structured like `new_model_trainer/configs/example_config.json`.
- It **must** contain a `dataset_items` list.
- Paths within `dataset_items` (`input_path`, `target_path`) can be either:
    - Relative to the `DRIVE_DATA_DIR` you specify below (e.g., `DI_audio/input1.wav`).
    - Absolute Google Drive paths (e.g., `/content/drive/MyDrive/MyAudio/input1.wav`).

In [ ]:
# Python code for Data Preparation cell

#@markdown ### (RUN CELL) 2.1: Specify Data and Configuration Location
#@markdown Mount your Google Drive if you haven't already (the Setup cell should do this).
#@markdown Then, provide the path to your main data directory on Google Drive and the name of your JSON configuration file within that directory.

DRIVE_DATA_DIR = "/content/drive/MyDrive/AidaX_Training_Data" #@param {type: "string"}
TRAINING_CONFIG_FILENAME = "my_training_run_config.json" #@param {type: "string"}

import os
import json
import shutil
import sys

# --- Define local Colab paths ---
COLAB_CONFIG_DIR = "/content/colab_configs"
COLAB_AUDIO_DIR = "/content/colab_training_audio"
LOCAL_TRAINING_CONFIG_PATH = os.path.join(COLAB_CONFIG_DIR, "effective_training_config.json")

os.makedirs(COLAB_CONFIG_DIR, exist_ok=True)
os.makedirs(COLAB_AUDIO_DIR, exist_ok=True)

print(f"User's Google Drive Data Directory: {DRIVE_DATA_DIR}")
print(f"User's Training Configuration Filename: {TRAINING_CONFIG_FILENAME}")

# --- Step 1: Copy and Parse the Main JSON Configuration File ---
drive_config_path = os.path.join(DRIVE_DATA_DIR, TRAINING_CONFIG_FILENAME)
local_temp_config_path = os.path.join(COLAB_CONFIG_DIR, "original_drive_config.json")

config_data = None # Initialize config_data
if not os.path.exists(drive_config_path):
    print(f"ERROR: Main training configuration file not found at: {drive_config_path}")
    sys.exit("Configuration file not found in Drive. Please check path and filename.") 
else:
    print(f"Copying main configuration file from {drive_config_path} to {local_temp_config_path}")
    shutil.copy(drive_config_path, local_temp_config_path)
    try:
        with open(local_temp_config_path, 'r') as f:
            config_data = json.load(f)
        print("Successfully loaded main configuration file.")
    except Exception as e:
        print(f"ERROR: Could not parse main training configuration file {local_temp_config_path}. Error: {e}")
        sys.exit("Failed to parse configuration file.")

if config_data:
    # --- Step 2: Process dataset_items, copy audio, and update paths ---
    if 'dataset_items' not in config_data or not isinstance(config_data['dataset_items'], list):
        print("ERROR: The configuration file must contain a 'dataset_items' list.")
        sys.exit("Invalid configuration: missing or malformed dataset_items.")
    if not config_data['dataset_items']:
        print("ERROR: 'dataset_items' list in configuration is empty. Nothing to process.")
        sys.exit("Empty dataset_items list.")

    updated_dataset_items = []
    all_files_copied_successfully = True
    
    # Clear out old audio data if any
    print(f"Clearing local audio directory: {COLAB_AUDIO_DIR}...")
    for item_name in os.listdir(COLAB_AUDIO_DIR):
        item_path = os.path.join(COLAB_AUDIO_DIR, item_name)
        try:
            if os.path.isfile(item_path) or os.path.islink(item_path):
                os.unlink(item_path)
            elif os.path.isdir(item_path):
                shutil.rmtree(item_path)
        except Exception as e:
            print(f'Failed to delete {item_path}. Reason: {e}')
    print(f"Local audio directory cleared.")

    for i, item in enumerate(config_data['dataset_items']):
        original_input_path = item.get('input_path')
        original_target_path = item.get('target_path')

        if not original_input_path or not original_target_path:
            print(f"WARNING: Skipping item {i} due to missing 'input_path' or 'target_path'.")
            all_files_copied_successfully = False # Mark as not fully successful
            continue

        full_drive_input_path = original_input_path if original_input_path.startswith("/content/drive/") else os.path.join(DRIVE_DATA_DIR, original_input_path)
        full_drive_target_path = original_target_path if original_target_path.startswith("/content/drive/") else os.path.join(DRIVE_DATA_DIR, original_target_path)

        if not os.path.exists(full_drive_input_path):
            print(f"ERROR: Input audio file not found for item {i}: {full_drive_input_path}")
            all_files_copied_successfully = False
            continue
        if not os.path.exists(full_drive_target_path):
            print(f"ERROR: Target audio file not found for item {i}: {full_drive_target_path}")
            all_files_copied_successfully = False
            continue
        
        input_basename = os.path.basename(original_input_path)
        target_basename = os.path.basename(original_target_path)
        local_input_filename = f"{i:03d}_input_{input_basename}"
        local_target_filename = f"{i:03d}_target_{target_basename}"
        
        local_colab_input_path = os.path.join(COLAB_AUDIO_DIR, local_input_filename)
        local_colab_target_path = os.path.join(COLAB_AUDIO_DIR, local_target_filename)

        try:
            # print(f"Copying input: {full_drive_input_path} -> {local_colab_input_path}")
            shutil.copy(full_drive_input_path, local_colab_input_path)
            # print(f"Copying target: {full_drive_target_path} -> {local_colab_target_path}")
            shutil.copy(full_drive_target_path, local_colab_target_path)
            
            updated_item = item.copy()
            updated_item['input_path'] = local_colab_input_path
            updated_item['target_path'] = local_colab_target_path
            updated_dataset_items.append(updated_item)
        except Exception as e:
            print(f"ERROR: Failed to copy files for item {i}. Input: {full_drive_input_path}, Target: {full_drive_target_path}. Error: {e}")
            all_files_copied_successfully = False
            continue # Skip this item
    
    if not updated_dataset_items:
        print("ERROR: No dataset items were successfully processed or all had errors. Please check paths and file existence in your Google Drive and config.")
        sys.exit("Dataset processing failed.")
    elif not all_files_copied_successfully:
        print("WARNING: Some files could not be copied or were missing. Proceeding with successfully processed items only.")

    config_data['dataset_items'] = updated_dataset_items
    
    model_name_sanitized = "".join(c if c.isalnum() else "_" for c in config_data.get('model_name', 'default_model'))
    local_output_dir = f"/content/training_output/{model_name_sanitized}"
    config_data['output_dir'] = local_output_dir 
    os.makedirs(local_output_dir, exist_ok=True)
    print(f"Output for training run will be in local Colab directory: {local_output_dir}")

    with open(LOCAL_TRAINING_CONFIG_PATH, 'w') as f:
        json.dump(config_data, f, indent=4)
    print(f"Effective training configuration with local paths saved to: {LOCAL_TRAINING_CONFIG_PATH}")
    print("\nData preparation cell complete. You can now proceed to 'Training Configuration'.")
    
    # Expose the path to the generated config for the next cells
    globals()['GLOBAL_COLAB_CONFIG_PATH'] = LOCAL_TRAINING_CONFIG_PATH
else:
    print("ERROR: Main configuration data not loaded in the first place. Cannot proceed.")
    globals()['GLOBAL_COLAB_CONFIG_PATH'] = None


## 3. Training Configuration

This cell allows you to make final adjustments to the training parameters. It loads the `effective_training_config.json` (prepared in the previous step, with localized data paths) and then applies any overrides you specify using the Colab form fields below.

The final configuration, including your overrides, will be saved back to `effective_training_config.json`, which is then used by the 'Run Training' cell.

In [ ]:
# Python code for Training Configuration cell

#@markdown ### (RUN CELL) 3.1: Adjust Training Parameters (Optional Overrides)
#@markdown This section allows you to override some common settings from your main configuration file (`{{TRAINING_CONFIG_FILENAME}}` loaded in the previous step).
#@markdown Leave fields blank or as their default to use the value from your JSON config.

#@markdown ---
#@markdown **Basic Settings:**
MODEL_NAME_OVERRIDE = "" #@param {type:"string"}
#@markdown Enter a specific number of epochs, or -1 to use the config file's value.
EPOCHS_OVERRIDE = -1 #@param {type:"integer"}
#@markdown Enter a specific batch size, or -1 to use the config file's value.
BATCH_SIZE_OVERRIDE = -1 #@param {type:"integer"}
#@markdown Enter a specific learning rate (e.g., 0.001), or 0.0 to use the config file's value.
LEARNING_RATE_OVERRIDE = 0.0 #@param {type:"number"}
DEVICE_OVERRIDE = "AUTO" #@param ["AUTO", "cuda", "cpu", "mps"]

#@markdown ---
#@markdown **Model Architecture (Overrides if specified):**
#@markdown LSTM Hidden Size, or -1 for config value.
HIDDEN_SIZE_OVERRIDE = -1 #@param {type:"integer"}
#@markdown Number of LSTM layers, or -1 for config value.
NUM_LAYERS_OVERRIDE = -1 #@param {type:"integer"}
#@markdown Skip Connection: "AUTO" (use config), "ON", "OFF".
SKIP_CONNECTION_OVERRIDE = "AUTO" #@param ["AUTO", "ON", "OFF"]

#@markdown ---
#@markdown **Advanced (Overrides if specified):**
#@markdown Early stopping patience, or -1 for config value.
EARLY_STOPPING_PATIENCE_OVERRIDE = -1 #@param {type:"integer"}


import json
import os
import sys

print("Applying Colab form overrides to the training configuration...")

effective_config_path = globals().get('GLOBAL_COLAB_CONFIG_PATH')

if not effective_config_path or not os.path.exists(effective_config_path):
    print(f"ERROR: Effective configuration file '{effective_config_path}' not found.")
    print("Please ensure the 'Data Preparation' cell (2.1) was run successfully and GLOBAL_COLAB_CONFIG_PATH was set.")
    sys.exit("Effective config not found, cannot apply overrides.")
else:
    with open(effective_config_path, 'r') as f:
        config_data = json.load(f)
    print(f"Loaded configuration from: {effective_config_path}")

    # Apply overrides from Colab form parameters
    if MODEL_NAME_OVERRIDE:
        original_model_name = config_data.get('model_name', 'default_model')
        config_data['model_name'] = MODEL_NAME_OVERRIDE
        # Update output_dir to reflect the new model name if it was based on the old one
        model_name_sanitized_original = "".join(c if c.isalnum() else "_" for c in original_model_name)
        expected_original_output_dir = f"/content/training_output/{model_name_sanitized_original}"
        
        model_name_sanitized_new = "".join(c if c.isalnum() else "_" for c in MODEL_NAME_OVERRIDE)
        new_output_dir = f"/content/training_output/{model_name_sanitized_new}"
        config_data['output_dir'] = new_output_dir
        os.makedirs(config_data['output_dir'], exist_ok=True)
        print(f"Overridden model_name to: {MODEL_NAME_OVERRIDE}")
        print(f"Updated output_dir to: {config_data['output_dir']}")

    if EPOCHS_OVERRIDE != -1:
        config_data['epochs'] = EPOCHS_OVERRIDE
        print(f"Overridden epochs to: {EPOCHS_OVERRIDE}")

    if BATCH_SIZE_OVERRIDE != -1:
        config_data['batch_size'] = BATCH_SIZE_OVERRIDE
        print(f"Overridden batch_size to: {BATCH_SIZE_OVERRIDE}")

    if LEARNING_RATE_OVERRIDE != 0.0:
        config_data['learning_rate'] = LEARNING_RATE_OVERRIDE
        print(f"Overridden learning_rate to: {LEARNING_RATE_OVERRIDE}")

    if DEVICE_OVERRIDE != "AUTO":
        config_data['device'] = DEVICE_OVERRIDE
        print(f"Overridden device to: {DEVICE_OVERRIDE}")
    
    if HIDDEN_SIZE_OVERRIDE != -1:
        config_data['hidden_size'] = HIDDEN_SIZE_OVERRIDE
        print(f"Overridden hidden_size to: {HIDDEN_SIZE_OVERRIDE}")
    
    if NUM_LAYERS_OVERRIDE != -1:
        config_data['num_layers'] = NUM_LAYERS_OVERRIDE
        print(f"Overridden num_layers to: {NUM_LAYERS_OVERRIDE}")

    if SKIP_CONNECTION_OVERRIDE != "AUTO":
        config_data['skip_connection'] = True if SKIP_CONNECTION_OVERRIDE == "ON" else False
        print(f"Overridden skip_connection to: {config_data['skip_connection']}")

    if EARLY_STOPPING_PATIENCE_OVERRIDE != -1:
        config_data['early_stopping_patience'] = EARLY_STOPPING_PATIENCE_OVERRIDE
        print(f"Overridden early_stopping_patience to: {EARLY_STOPPING_PATIENCE_OVERRIDE}")

    with open(effective_config_path, 'w') as f:
        json.dump(config_data, f, indent=4)
    print(f"Final configuration saved to: {effective_config_path}")
    print(f"To be used by train_model.py: Model name '{config_data.get('model_name')}', Output dir '{config_data.get('output_dir')}'")
    print("\nTraining Configuration cell complete.")

    globals()['GLOBAL_COLAB_CONFIG_PATH'] = effective_config_path # Ensure it's correctly set for the next cell


## 4. Run Training

This cell executes the `train_model.py` script using the `effective_training_config.json` that was prepared and potentially modified in the previous steps. 

Training progress, including loss values and epoch information, will be displayed in the output of this cell. Depending on the number of epochs, dataset size, and selected device (CPU/GPU), this can take a considerable amount of time.

In [ ]:
# Python code for Run Training cell

#@markdown ### (RUN CELL) 4.1: Start Model Training
#@markdown This cell will execute the `train_model.py` script using the configuration prepared in the previous steps.
#@markdown Training progress will be displayed below. This may take a significant amount of time depending on your data and settings.

import os
import json # For loading the config to show output dir
import sys

# GLOBAL_COLAB_CONFIG_PATH should have been set by the 'Training Configuration' cell.
final_config_path = globals().get('GLOBAL_COLAB_CONFIG_PATH')

if not final_config_path or not os.path.exists(final_config_path):
    print(f"ERROR: Final configuration file '{final_config_path}' not found!")
    print("Please ensure the 'Data Preparation' (2.1) and 'Training Configuration' (3.1) cells were run successfully.")
    sys.exit("Final config not found. Cannot start training.")
else:
    print(f"Starting training using configuration: {final_config_path}")
    
    # Ensure we are in the new_model_trainer directory where train_model.py is located.
    # The setup cell (1.1) should have handled changing to the correct directory.
    # Adding a check here for robustness.
    expected_cwd_suffix = "new_model_trainer"
    if not os.getcwd().endswith(expected_cwd_suffix):
        print(f"WARNING: Current working directory is {os.getcwd()}, not ending with '{expected_cwd_suffix}'.")
        # Attempt to change to the correct directory if the repo was cloned in /content
        repo_base_path = "/content/Automated-GuitarAmpModelling"
        trainer_path = os.path.join(repo_base_path, expected_cwd_suffix)
        if os.path.exists(trainer_path):
            print(f"Attempting to change directory to: {trainer_path}")
            os.chdir(trainer_path)
            print(f"New current working directory: {os.getcwd()}")
        else:
            print(f"ERROR: Cannot automatically find or change to directory '{trainer_path}'. Training might fail if train_model.py is not found.")

    # Using '!' to run the shell command and see output directly in Colab
    # The output from train_model.py (print statements, tqdm progress bars) will appear here.
    get_ipython().system(f'python train_model.py --config_file="{final_config_path}"')

    print("\n--- Training Run Attempted ---")
    print(f"If training started, check the output above for progress and completion status.")
    
    # Try to load the config again to get the output_dir and model_name for user feedback
    try:
        with open(final_config_path, 'r') as f:
            final_config_data = json.load(f)
        output_dir_from_config = final_config_data.get('output_dir')
        model_name_from_config = final_config_data.get('model_name')

        if output_dir_from_config and model_name_from_config:
            print(f"Results (checkpoints, logs, .aidax model) should be in: {output_dir_from_config}")
            if os.path.exists(output_dir_from_config):
                print(f"\nContents of the output directory ({output_dir_from_config}):")
                for item in os.listdir(output_dir_from_config):
                    print(f"- {item}")
                # Check for .aidax file specifically
                expected_aidax_path = os.path.join(output_dir_from_config, f"{model_name_from_config}.aidax")
                if os.path.exists(expected_aidax_path):
                    print(f"\nSUCCESS: Exported model found at {expected_aidax_path}")
                else:
                    print(f"\nNOTE: Exported .aidax model not immediately found at {expected_aidax_path}. It might be in a subdirectory if model_name in config was different, or training/export might have failed.")
            else:
                print(f"\nNOTE: Output directory {output_dir_from_config} specified in config was not found after training attempt.")
        else:
            print("\nNOTE: Could not determine output_dir or model_name from config to list results.")
    except Exception as e:
        print(f"Note: Could not list output directory contents due to an error: {e}")


## 5. Export & Download Model

After training, the `train_model.py` script automatically exports the best model to the `.aidax` format. This cell helps you locate that file and provides options to download it to your local machine or copy it to a specified folder in your Google Drive.

In [ ]:
# Python code for Export & Download Model cell

#@markdown ### (RUN CELL) 5.1: Access Your Trained Model
#@markdown This cell helps you locate the exported `.aidax` model file and provides options to download it or copy it to your Google Drive.
#@markdown The `.aidax` file should have been generated automatically at the end of a successful training run.

#@markdown ---
#@markdown **Specify Google Drive Destination (Optional):**
#@markdown If you want to copy the `.aidax` file to a specific folder in your Google Drive, enter the path below relative to your Drive's root.
#@markdown Example: `MyAidaModels/Exported` (This will be created under `/content/drive/MyDrive/`).
#@markdown Leave blank if you only want to download it locally.
GDRIVE_EXPORT_SUBDIR = "AidaX_Models_Exported" #@param {type:"string"}

import os
import json
import shutil
import sys
from google.colab import files # For downloading

print("Locating the exported model...")
final_config_path = globals().get('GLOBAL_COLAB_CONFIG_PATH')

if not final_config_path or not os.path.exists(final_config_path):
    print(f"ERROR: Final configuration file '{final_config_path}' not found!")
    print("Please ensure the entire notebook has been run successfully up to the training step.")
    sys.exit("Final config not found.")
else:
    try:
        with open(final_config_path, 'r') as f:
            config_data = json.load(f)
        
        model_name = config_data.get('model_name')
        local_output_dir = config_data.get('output_dir') # This is the local Colab path

        if not model_name or not local_output_dir:
            print("ERROR: 'model_name' or 'output_dir' not found in the configuration.")
            sys.exit("Missing critical config info for locating model.")
        
        expected_aidax_filename = f"{model_name}.aidax"
        local_aidax_path = os.path.join(local_output_dir, expected_aidax_filename)

        if os.path.exists(local_aidax_path):
            print(f"SUCCESS: Exported model found at: {local_aidax_path}")
            print(f"File size: {os.path.getsize(local_aidax_path) / 1024:.2f} KB")

            # Option 1: Download the file
            print("\n--- Option 1: Download to your local machine ---")
            try:
                files.download(local_aidax_path)
                print(f"Download initiated for {expected_aidax_filename}.")
            except Exception as e:
                print(f"Could not initiate download. Error: {e}")
                print(f"You can manually download it from the Colab file browser (left panel), under: {local_aidax_path}")

            # Option 2: Copy to Google Drive
            if GDRIVE_EXPORT_SUBDIR:
                drive_base_path = "/content/drive/MyDrive"
                full_gdrive_export_path = os.path.join(drive_base_path, GDRIVE_EXPORT_SUBDIR)
                print(f"\n--- Option 2: Copy to Google Drive --- ")
                print(f"Target Google Drive folder: {full_gdrive_export_path}")
                
                if not os.path.exists(full_gdrive_export_path):
                    print(f"Google Drive destination path '{full_gdrive_export_path}' does not exist. Attempting to create it.")
                    try:
                        os.makedirs(full_gdrive_export_path, exist_ok=True)
                        print(f"Created Google Drive directory: {full_gdrive_export_path}")
                    except Exception as e:
                        print(f"ERROR: Could not create directory on Google Drive: {full_gdrive_export_path}. Error: {e}")
                        # Prevent copy attempt if directory creation failed
                        full_gdrive_export_path = None 
                
                if full_gdrive_export_path and os.path.exists(full_gdrive_export_path):
                    drive_destination_file_path = os.path.join(full_gdrive_export_path, expected_aidax_filename)
                    try:
                        shutil.copy(local_aidax_path, drive_destination_file_path)
                        print(f"Model successfully copied to Google Drive: {drive_destination_file_path}")
                    except Exception as e:
                        print(f"ERROR: Failed to copy model to Google Drive. Error: {e}")
                elif full_gdrive_export_path: # Only print if path was attempted
                    print(f"Skipping copy to Google Drive as destination path '{full_gdrive_export_path}' could not be confirmed/created.")
            else:
                print("\n--- Option 2: Copy to Google Drive ---")
                print("No Google Drive destination subdirectory specified. Skipping copy to Drive.")
        else:
            print(f"ERROR: Exported model file not found at expected location: {local_aidax_path}")
            print("This might indicate that the training or export step failed, was not completed, or the model_name/output_dir in your config differs from the training run.")
            print(f"Please check the output of the 'Run Training' cell and the contents of the expected output directory: {local_output_dir}")

    except Exception as e:
        print(f"An error occurred while trying to access the model for export/download: {e}")
        print("Ensure previous cells, especially training, completed successfully and defined 'GLOBAL_COLAB_CONFIG_PATH'.")

print("\nExport & Download Model cell execution finished.")